In [61]:
!pip install --upgrade torch==1.7.1 torchtext==0.8.1 torchvision==0.8.2

In [62]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

2021-05-01 11:12:19.310630: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 13.7 MB 98 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [63]:
import torch, torchtext, numpy as np
import pandas as pd, csv
from torch import nn, optim
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
#import pdbf

In [64]:
from google.colab import files
uploaded = files.upload()

In [120]:
!unzip archive.zip

Archive:  archive.zip
replace test.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  archive.zip
replace test.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: test.txt                
  inflating: train.txt               
  inflating: val.txt                 


In [101]:
#dataset_full = pd.read_csv('text_dataset.csv')

In [123]:
#dataset_full

In [205]:
data_train_csv = pd.read_csv('train.txt', sep=';', names = ['caption', 'sentiment'])
data_train_csv.to_csv('train.csv')

In [206]:
data_train_csv

,caption,sentiment
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
15995,i just had a very brief time in the beanbag an...,sadness
15996,i am now turning and i feel pathetic that i am...,sadness
15997,i feel strong and good overall,joy
15998,i feel like this was such a rude comment and i...,anger


In [207]:
data_test_csv = pd.read_csv('test.txt', sep=';', names = ['caption', 'sentiment'])
data_test_csv.to_csv('test.csv')

In [208]:
data_test_csv

,caption,sentiment
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness
...,...,...
1995,i just keep feeling like someone is being unki...,anger
1996,im feeling a little cranky negative after this...,anger
1997,i feel that i am useful to my people and that ...,joy
1998,im feeling more comfortable with derby i feel ...,joy


In [209]:
data_validation_csv = pd.read_csv('val.txt', sep=';', names = ['caption', 'sentiment'])
data_validation_csv.to_csv('val.csv')

In [210]:
data_validation_csv

,caption,sentiment
0,im feeling quite sad and sorry for myself but ...,sadness
1,i feel like i am still looking at a blank canv...,sadness
2,i feel like a faithful servant,love
3,i am just feeling cranky and blue,anger
4,i can have for a treat or if i am feeling festive,joy
...,...,...
1995,im having ssa examination tomorrow in the morn...,sadness
1996,i constantly worry about their fight against n...,joy
1997,i feel its important to share this info for th...,joy
1998,i truly feel that if you are passionate enough...,joy


Now joining all the 3 given databases into one

In [211]:
complete_dataset_csv = data_train_csv
complete_dataset_csv = complete_dataset_csv.append(data_test_csv)
complete_dataset_csv = complete_dataset_csv.append(data_validation_csv)
complete_dataset_csv.to_csv('complete_dataset.csv')
complete_dataset_csv.size

40000

Emotions before editing:

In [238]:
complete_dataset_csv.sentiment.unique()

array(['2', '0', '3', '1'], dtype=object)

Mapping:


0.   sad
1.   anger
2.   fear
3.   happy



In [239]:
complete_dataset_csv['sentiment'] = complete_dataset_csv['sentiment'].replace(['sadness','joy','anger','fear'],['0','3','1','2'])

1-> angry, 3-> happy, 0-> sad, 2->fear

In [240]:
complete_dataset_csv[:10]

,caption,sentiment
0,i didnt feel humiliated,2
1,i can go from feeling so hopeless to so damned...,2
2,im grabbing a minute to post i feel greedy wrong,0
4,i am feeling grouchy,0
5,ive been feeling a little burdened lately wasn...,2
7,i feel as confused about life as a teenager or...,3
8,i have been with petronas for years i feel tha...,1
10,i feel like i have to make the suffering i m s...,2
11,i do feel that running is a divine experience ...,1
12,i think it s the easiest time of year to feel ...,0


In [241]:
complete_dataset_csv = complete_dataset_csv[complete_dataset_csv.sentiment != 'surprise']
complete_dataset_csv = complete_dataset_csv[complete_dataset_csv.sentiment != 'love']


In [242]:
complete_dataset_csv.sentiment.unique()

array(['2', '0', '3', '1'], dtype=object)

In [243]:
complete_dataset_csv.size

35280

In [244]:
complete_dataset_csv.to_csv('complete_dataset_final_2.csv')

sentiments: 0 for angry; 1 for happy; 2 for sad; 3 for sleepy

In [245]:
class dataset_cat_sentiments(torch.utils.data.Dataset):

  def __init__(self, csv_data, vocab=None):
    self.csv_dataset = pd.read_csv(csv_data)
    self.tokenizer = torchtext.data.utils.get_tokenizer('spacy', 'en_core_web_sm')
    toks = self.tokenizer(" ".join(self.csv_dataset.caption.unique()))

    if vocab is None:
            self.vocab = torchtext.vocab.build_vocab_from_iterator([toks])
    else:
            self.vocab = vocab
    self.toks = torch.LongTensor([self.vocab[tok] for tok in toks])
    self.LABELS = self.csv_dataset.sentiment.unique()
    self.LENGTH = self.csv_dataset.shape[0]
    self.label_dict = dict(zip(self.LABELS,range(len(self.LABELS))))

    self.dataset_dict = []
    for i in range(self.LENGTH):
      labels_ret = self.label_dict[self.csv_dataset.iloc[i].sentiment]
      toks = self.tokenizer(self.csv_dataset.iloc[i].caption)
      toks_ret = torch.LongTensor([self.vocab[tok] for tok in toks])
      self.dataset_dict.append((labels_ret, toks_ret))
    
    
  def get_labels(self):
    return set([(i) for i in range(len(self.LABELS))])

  def get_vocab(self):
    return self.vocab

  def DATASETS(self):
    return self.dataset_dict[:int(self.LENGTH*8/10)], self.dataset_dict[int((self.LENGTH*(8/10))+1):]

  def __len__(self):
    return self.LENGTH

  def __getitem__(self, i):
    
    # labels_ret = self.label_dict[self.csv_dataset.iloc[i].sentiment]
    # toks = self.tokenizer(self.csv_dataset.iloc[i].text)
    # toks_ret = torch.LongTensor([self.vocab[tok] for tok in toks])
    # return (labels_ret, toks_ret)
    return self.dataset_dict[i]

In [246]:
ap = dataset_cat_sentiments('complete_dataset_final.csv')
ap.LABELS
ap.get_labels()
ap

1lines [00:00, 20.63lines/s]


In [247]:
train_ds, test_ds = ap.DATASETS()

In [248]:
vb = ap.get_vocab()
len(vb)
vb.itos[:10]

['<unk>', '<pad>', 'i', 'feel', 'and', 'to', 'the', 'a', 'feeling', 'that']

In [249]:
vb.stoi['should']

114

In [250]:
xs, ys = ap[0]
' '.join([ap.vocab.itos[x] for x in ys])

'i did nt feel humiliated'

In [251]:
def mk_offsets(samples):
    txts = [txt for _, txt in samples]
    lens = [0] + [len(txt) for txt in txts]
    offs = torch.tensor(lens[:-1]).cumsum(0)
    labs = torch.tensor([lab for lab, txt in samples])
    txts = torch.cat(txts)
    return labs, txts, offs

In [252]:
mk_offsets(test_ds[0:4])

(tensor([3, 0, 3, 0]),
 tensor([   2,    3, 1685,   21,    2,  166,    9,    2,   23,   91,   40, 1711,
            4,  577,    2,    3,  508,   38,    2,   14,   78,  789,    2,    3,
           44,  309,    9,    2,   30,   29,  160,   15,    6, 8813,   50,   80,
            9,   45,   16,  613,   92,  303, 2097,    2,   19,  266,    5,  805,
         3238,   11,  126,   10,    7,  463,  415,   10,   11,  389,    4,   19,
          509,   13,   38,    2,   19,  193,  126,   12,    6,  463, 6441]),
 tensor([ 0, 14, 22, 43]))

In [253]:
class TextCats(nn.Module):
    def __init__(self, n_words, emb_dim, n_cats):
        super().__init__()
        self.embedding = nn.EmbeddingBag(n_words, emb_dim, mode='mean', sparse=True)
        self.fc = nn.Linear(emb_dim, n_cats)
        nn.init.xavier_uniform_(self.embedding.weight.data)
        nn.init.xavier_uniform_(self.fc.weight.data)
    def forward(self, text, offs):
        emb = self.embedding(text, offs)
        return self.fc(emb)

In [254]:
device = torch.device('cpu')

def run_test(model, ds, crit):
    model.eval()
    total_loss, total_acc = 0, 0
    ldr = torch.utils.data.DataLoader(ds, batch_size=16, collate_fn=mk_offsets)
    for labs, txts, offs in tqdm(ldr, leave=False, desc='test iter'):
        labs, txts, offs = labs.to(device), txts.to(device), offs.to(device)
        with torch.no_grad():
            outs = model(txts, offs)
            loss = crit(outs, labs)
            total_loss += loss.item()
            total_acc += (outs.argmax(1) == labs).sum().item()
    return total_loss / len(ds), total_acc / len(ds)

def run_train(model, ds, crit, opt, sched):
    model.train()
    total_loss, total_acc = 0, 0
    ldr = torch.utils.data.DataLoader(ds, batch_size=16, collate_fn=mk_offsets, shuffle=True)
    for labs, txts, offs in tqdm(ldr, leave=False, desc='train iter'):
        opt.zero_grad()
        labs, txts, offs = labs.to(device), txts.to(device), offs.to(device)
        outs = model(txts, offs)
        loss = crit(outs, labs)
        loss.backward()
        opt.step()
        total_loss += loss.item()
        total_acc += (outs.argmax(1) == labs).sum().item()
    sched.step()
    return total_loss / len(ds), total_acc / len(ds)

def run_all(model, test_ds, train_ds, crit, opt, sched, n_epochs=10):
    for epoch in tqdm(range(n_epochs), desc='epochs'):
        train_loss, train_acc = run_train(model, train_ds, crit, opt, sched)
        test_loss, test_acc = run_test(model, test_ds, crit)
        tqdm.write(f'epoch {epoch}   train loss {train_loss:.6f} acc {train_acc:.4f}   test loss {test_loss:.6f} acc {test_acc:.4f}') 

In [255]:
model = TextCats(len(vb), 50, 4)
model.to(device);

In [256]:
crit = nn.CrossEntropyLoss().to(device)
opt = optim.SGD(model.parameters(), lr=3.0)
sched = optim.lr_scheduler.StepLR(opt, 1, gamma=0.7)

In [257]:
run_all(model, test_ds, train_ds, crit, opt, sched, 10) 

epoch 0   train loss 0.070935 acc 0.4972   test loss 0.038061 acc 0.7783


epoch 1   train loss 0.028958 acc 0.8403   test loss 0.027437 acc 0.8299


epoch 2   train loss 0.014663 acc 0.9308   test loss 0.018484 acc 0.9067


epoch 3   train loss 0.009902 acc 0.9573   test loss 0.016046 acc 0.9220


epoch 4   train loss 0.007547 acc 0.9705   test loss 0.014336 acc 0.9215


epoch 5   train loss 0.006378 acc 0.9792   test loss 0.013692 acc 0.9291


epoch 6   train loss 0.005584 acc 0.9823   test loss 0.013364 acc 0.9303


epoch 7   train loss 0.005121 acc 0.9855   test loss 0.013520 acc 0.9305


epoch 8   train loss 0.004825 acc 0.9877   test loss 0.013412 acc 0.9277


epoch 9   train loss 0.004627 acc 0.9896   test loss 0.013413 acc 0.9291




Testing out the model:
---



1-> angry, 3-> happy, 0-> sad, 2->fear

In [262]:
lst = ["i am felling grouchy"]
str_arr = (lst[0].split())
str_index = []
str_index2 = []
str_index3 = []
for i in range(len(str_arr)):
  str_index.append(vb.stoi[str_arr[i]])
  
  if(i+1<len(str_arr)):
    str_index2.append(vb.stoi[str_arr[i]+' '+str_arr[i+1]])
  
tensor_in = torch.LongTensor(str_index)
model.eval()
outs = model(tensor_in, torch.LongTensor([0]))
outs.argmax(1)
# vb.stoi['hello', 'cat', 'is', 'very', 'sad']

tensor([1])

Optimizing our model for mobile and downloading it:


In [264]:
from torch.utils.mobile_optimizer import optimize_for_mobile

In [265]:
MemeSentiment_model = torch.jit.script(model)

In [266]:
torchscript_model_optimized = optimize_for_mobile(MemeSentiment_model)
torch.jit.save(torchscript_model_optimized, "MemeSentiment_model.pt")